# Intelligent Agents: Reflex-Based Agents for GridHunt

Student Name: [Add your name]

I have used the following AI tools: [list tools]

I understand that my submission needs to be my own work: [your initials]

## Learning Outcomes

* Apply core AI concepts by implementing the agent function for a simple and model-based reflex agents that respond to environmental percepts.
* Practice how the environment and the agent function interact.
* Analyze agent performance through controlled experiments across different environment configurations.

## Instructions

Total Points: Undergrads 10

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook.
### AI Use

Here are some guidelines that will make it easier for you:

* __Don't:__ Rely on AI auto completion. You will waste a lot of time trying to figure out how the suggested code relates to what we do in class. Turn off AI code completion (e.g., Copilot) in your IDE.
* __Don't:__ Do not submit code/text that you do not understand or have not checked to make sure that it is complete and correct.
* __Do:__ Use AI for debugging and letting it explain code and concepts from class.

## Introduction

Gridhunt-Agent is a small educational game to play with implementing simple agent functions. It is a Python reimplementation based on [Gridhunt2](https://michael.hahsler.net/SMU/CS1342/gridhunt2/). 

The objective of gridhunt is to implement a hunter who can catch the monster faster than all other hunters. Gridhunt uses a 30x30 grid of squares and a monster moves on this grid. Gridhunt is turn-based. In each turn each hunter gets 1-5 points which she/he can use for actions (e.g., move to an adjacent square, find out where the monster is; teleport to a random location on the grid). A hunter catches the monster if she/he moves on the same square the monster currently occupies first. If the monster survives 100 rounds the monster wins.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to catch the monster. Catching means to be on the same square.

__Environment:__ An arena with $30 \times 30$ squares. At the beginning the monster and the player is randomly placed in the environment.

__Actuators:__ The agent can move to an adjacent square or teleport which will move the agent to a random square in the arena.

__Sensors:__ The agent can see its location in the arena and the location of the monster.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [ ]:
import numpy as np
from IPython.display import clear_output
from time import sleep

def arena_environment(hunter_agent_function, monster_agent_function, n = 30, max_steps = 100, visualize = False, animation = False):
    """
    Simulate an arena where a hunter agent tries to catch a monster agent.

    Parameters:
    hunter_agent_function (function): A function that takes the current positions of the hunter and monster
                                      and returns the next move for the hunter.
    monster_agent_function (function): A function that takes the current positions of the hunter and monster
                                       and returns the next move for the monster
    n (int): The size of the arena (n x n grid).
    max_steps (int): The maximum number of steps to simulate.
    visualize (bool): Whether to visualize the arena.
    animation (bool): Whether to animate the visualization with a delay.

    Returns:
    int: The number of steps taken for the hunter to catch the monster or max_steps if not caught.
    """
    
    # Initialize positions
    monster_pos = np.random.randint(0, n-1, size=2)
    hunter_pos = np.random.randint(0, n-1, size=2)
    
    def move(action, position):
        """calculate new position for the agent."""
        if action == 'north':
            position[0] -= 1
        elif action == 'south':
            position[0] += 1
        elif action == 'west':
            position[1] -= 1
        elif action == 'east':
            position[1] += 1
        else:
            raise ValueError("Invalid action.")
        
        # Ensure position stays within bounds
        position = np.clip(position, 0, n-1)
        return position

    for step in range(max_steps):
        if visualize:
            if animation:
                sleep(1)
                clear_output(wait=True)
            arena = np.full((n, n), '.', dtype=str)
            arena[monster_pos[0], monster_pos[1]] = 'M'
            arena[hunter_pos[0], hunter_pos[1]] = 'H'
            print("\n".join("".join(row) for row in arena))
        
        
        if np.array_equal(hunter_pos, monster_pos):
            print(f"Hunter caught the monster in {step} steps!")
            break
        
        # Get next move from monster agent function
        monster_action = monster_agent_function(hunter_pos, monster_pos)
        if monster_action != 'stay':
            monster_pos = move(monster_action, monster_pos)

        # Get next move from hunter agent function
        hunter_action = hunter_agent_function(hunter_pos, monster_pos)
        if hunter_action == 'teleport':
            hunter_pos = np.random.randint(0, n-1, size=2)
        else:
            hunter_pos = move(hunter_action, hunter_pos)
    
        if visualize:
            print(f"Hunter chose action '{hunter_action}' and is now at {hunter_pos}")
            print(f"Monster chose action '{monster_action}' and is now at {monster_pos}")
            print("\n" + "="*n + "\n")
        
    return step

In [14]:
monster_actions = ["north", "east", "west", "south", "stay"]

def monster_agent_function_simple(hunter_pos, monster_pos):
    return np.random.choice(monster_actions, p=[0.125, 0.125, 0.125, 0.125, 0.5])

In [15]:
actions = ["north", "east", "west", "south", "teleport"]

def simple_randomized_hunter_agent_function(hunter_location, monster_location):
    return np.random.choice(actions)

In [ ]:
arena_environment(simple_randomized_hunter_agent_function, monster_agent_function_simple, n=5, max_steps=5, visualize=True)

..H..
.....
.....
.....
....M
Hunter chose action 'east' and is now at [0 3]
Monster chose action 'south' and is now at [4 4]

=====



KeyboardInterrupt: 

In [41]:
arena_environment(simple_randomized_hunter_agent_function, monster_agent_function_simple, n=5, max_steps=100, visualize=False)

Hunter caught the monster in 14 steps!


14